In [ ]:
%%writefile /content/drive/MyDrive/rag/models/V3(ensemble).py

Writing /content/drive/MyDrive/rag/models/pipeline.py


rag 품질 개선

1st 시도:\
child parent retriever\
기존의 rag와 큰 품질 차이 없음\
뭐가 문제일까\
\
2nd 시도:\
multi query retriever\
국세청 자주 묻는 질문을 보면 세법 전문지식을 가진 사람들의 질문이 아니라서 모호한 질문이 많다.\
따라서 prompt를 통해 질문을 구체화시킴\
여전히 rag 품질 문제 - 답변과 논리적 근거의 비일관성. retrieved 문서의 부적합함\
\
3rd 시도:\
bm25를 이용한 hybrid retriever\
multi query 유지(명시적 구현), llm이 답변 생성시 metadata도 함께 보도록 함

In [1]:
try:
  from langchain_community.retrievers import BM25Retriever
except ImportError:
  !pip install langchain_community
  !pip install rank_bm25
  !pip install faiss-cpu

try:
  from langchain_core.prompts import PromptTemplate
except ImportError:
  !pip install langchain_core
  from langchain_core.prompts import PromptTemplate

try:
  from langchain_openai import ChatOpenAI
except ImportError:
  !pip install langchain_openai
  from langchain_openai import ChatOpenAI

try:
  import chromadb
except ImportError:
  !pip install chromadb
  import chromadb

try:
  from langchain_chroma import Chroma
except ImportError:
  !pip install langchain_chroma
  from langchain_chroma import Chroma

try:
  from langchain_openai import OpenAIEmbeddings
except ImportError:
  !pip install langchain_openai
  from langchain_openai import OpenAIEmbeddings

try:
  from langchain_core.documents import Document
except ImportError:
  !pip install langchain_core
  from langchain_core.documents import Document

try:
  from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
except ImportError:
  !pip install langchain_core
  from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

import json
import re

In [2]:
try:
  from langchain_classic.retrievers import EnsembleRetriever
except ImportError:
  !pip install langchain_classic
  from langchain_classic.retrievers import EnsembleRetriever

In [3]:
from google.colab import userdata

llm = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0,
    max_tokens = 1000,
    openai_api_key = userdata.get('OPENAI_API_KEY')
)

db_path = "/content/drive/MyDrive/rag/chromadb_backup"
client = chromadb.PersistentClient(path=db_path)
collection_name = "tax_law"

try:
  collection = client.get_collection(name=collection_name)
except Exception:
  raise RuntimeError(f"Collection {collection_name} not found")

embedding_model = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embedding_model, openai_api_key = userdata.get("OPENAI_API_KEY"))

vectorstore = Chroma(client = client, collection_name=collection_name, embedding_function=embeddings)


In [4]:
class LLMReranker:

  def __init__(self, llm, top_k=7):
    self.llm = llm
    self.top_k = top_k
    self.prompt_reranking = """
    너는 세법 전문가야. 질문의 의도에 가장 적절한 답변을 제공하는 상위 {top_k}개의 문서들을 선별해.

    - 질문에 포함된 수치가 단순히 언급된 것이 아니라, 질문이 요구하는 '논리적 정의', '조건', '한도', '계산근거'로서 설명되는 문서를 우선시해.
    - 질문과 무관한 법령은 단어가 겹쳐도 낮은 점수를 줘.
    - 문서 간 상대적 우위를 비교하여 상위 {top_k}개의 문서들에 대해 0~100점 사이로 차등 점수를 줘.

    [질문]
    {question}

    [문맥]
    {context}

    [출력 형식 - JSON]
    {{
      "scores": [
        {{"id": 0, "score": 93, "reason": "법적 조건인 연간 소득금액 100만원에서 소득금액의 정의를 구체적으로 명시함"}},
        {{"id": 3, "score": 65, "reason": "연간 소득금액 100만원의 합산 범위를 구체적으로 명시함"}},
        ...
      ]
    }}
    """
    self.prompt = PromptTemplate(input_variables=["question", "context", "top_k"], template=self.prompt_reranking)

  def rerank(self, docs, query):

    docs_matching = {str(i): doc for i, doc in enumerate(docs)}

    docs_formatted = "\n".join([
        f"[ID {i}] ([출처] law name: {doc.metadata.get('law_name')}, 조문제목: {doc.metadata.get("조문제목")}) {doc.page_content[:500]}"
        for i, doc in enumerate(docs)])

    chain = self.prompt | self.llm | JsonOutputParser()

    rerank_result = chain.invoke({"question": query, "context": docs_formatted, "top_k": self.top_k})
    scores = rerank_result.get("scores", [])

    scored_docs = []
    for x in scores:
      id = str(x.get("id"))
      score = int(x.get("score", 0))
      if id in docs_matching:
        scored_docs.append((docs_matching[id], score))

    scored_docs.sort(key=lambda x: x[1], reverse = True)

    reraked_docs = [doc for doc, _ in scored_docs[:self.top_k]]

    return reraked_docs



[multi query로 변환
prompt에게 전문용어로 변경하도록 함\
\
bm25와 basic retriever를 합친 ensemble retriever 사용\
\
reciprocal rank fusion (rrf) 사용해 각 query에 대한 결과 합산\
\
llm 기반 re ranker로 상위 final_k개 정렬]\
\
일반적으로 retriever를 여러개 사용할 때 서로 다른 검색기로부터 검색된 문서들을 정렬하기 위해 rrf가 사용됨\
하지만 내 경우에는 ensemble retriever를 사용하기 때문에 여러 개의 retriever가 찾은 문서들이 내가 정한 weights에 따라 가중평균되어 정렬됨\
오히려 multi query 변환으로 인해 각 query에 따른 ensemble retriever의 문서들을 정렬하는 문제에 rrf 사용함.\
(앞쪽 query에 편향되는 것을 막고, 어떤 관점(query)으로 검색해도 중요한 문서를 llm reranker에게 우선적으로 넘겨줌)\
\
layer1: ensemble\
각 쿼리에 대해 키워드와 의미가 적절히 섞인 가장 균형 잡힌 15개를 뽑아냄\
layer2: rrf\
여러 쿼리에서 나온 후보들 중 반복적으로 등장하는 가장 신뢰도 높은 20개 추려냄\
layer3: llm rerank\
최종 후보들 중 최적의 5개 결정

In [5]:
class Pipeline:

  def __init__(self, query, llm, vectorstore, initial_k=15, final_k=20, top_k=7):
    self.initial_k = initial_k
    self.final_k = final_k
    self.top_k = top_k
    self.query = query
    self.llm = llm
    self.vectorstore = vectorstore

    all_data = vectorstore.get()
    all_docs = [
        Document(page_content= content, metadata=metadata)
        for content, metadata in zip(all_data['documents'], all_data['metadatas'])
    ]

    self.bm25_retriever = BM25Retriever.from_documents(all_docs, search_kwargs={"k": self.initial_k})
    self.vector_retriever = vectorstore.as_retriever(search_kwargs={"k": self.initial_k})
    self.ensemble_retriever = EnsembleRetriever(retrievers = [self.bm25_retriever, self.vector_retriever], weights = [0.3, 0.7])

    self.reranker = LLMReranker(llm, self.top_k)


  def generate_queries(self):
    prompt_multi_query = """
    사용자의 질문을 분석해서 다음 3가지 관점의 세법 전문가용 검색어로 변환해줘
    [요건 관점] 질문의 상황과 관련된 법령 적용의 직접적인 요건 (ex. 부양가족 공제 소득요건)
    [정의 관점] 질문에 포함된 핵심 용어의 법적 정의 (ex. 소득세법상 소득금액의 정의 및 범위)
    [계산 관점] 적되는 산식, 계산 방법

    실제 법령과 집행기준, 판례에 쓰이는 전문 용어를 사용해.

    [질문]
    {question}

    [출력형식]
    검색어를 줄바꿈만으로 구분
    """

    prompt = PromptTemplate(input_variables=["question"], template=prompt_multi_query)
    chain = prompt | self.llm | StrOutputParser()
    multi_query = chain.invoke({"question": self.query})
    multi_query = multi_query.split("\n")
    multi_query.append(self.query)
    return multi_query

  def retrieve(self):
    multi_query = self.generate_queries()

    k_const = 60

    docs_score = {}
    docs_list = {}
    for query in multi_query:
      docs = self.ensemble_retriever.invoke(query)
      for rank, doc in enumerate(docs):
        content = doc.page_content

        score = 1.0/(k_const + rank)
        if content in docs_score:
          docs_score[content] += score
        else:
          docs_score[content] = score
          docs_list[content] = doc

    sorted_docs = sorted(docs_score.items(), key=lambda x: x[1], reverse=True)
    sorted_list = [docs_list[content] for content, _ in sorted_docs[:self.final_k]]

    reranked_docs = self.reranker.rerank(sorted_list, self.query)

    return reranked_docs

  def generate_answer(self):
    reranked_docs = self.retrieve()

    context = "\n\n".join([
        f"[ID {i}] ([출처] law name: {doc.metadata.get('law_name')}, 조문제목: {doc.metadata.get('조문제목')})\n{doc.page_content[:500]}"
        for i, doc in enumerate(reranked_docs)
    ])

    # print(context)

    prompt_final = """
    너는 세법 전문가야. 아래의 문맥을 근거로 질문에 답변해.

    [규칙]
    1. 질문에 대한 답은 한 문장으로 먼저 제시 (ex. "납부할 세액은 1,000,000원입니다." "해당 항목은 익금산입 대상입니다.")
    2. 답변 근거가 된 문장 끝에 반드시 해당 문서 ID인 [[번호]]를 기입할 것.
    3. used_index는 답변 근거에 등장한 [[번호]]만 그대로 추출하여 리스트로 만들 것.
    4. 답변시 법령의 예외 조항이나 단서 조항(다만, ~의 경우 등)이 있는지 확인해서 반영할 것.
    5. 문맥이 질문에 대한 답변 근거를 제공하지 못하면, "제공된 자료에서 관련 내용을 찾을 수 없습니다"로 답하고 used_index는 빈 리스트 [] 제공.

    [문맥]
    {context}

    [질문]
    {question}

    [출력 형식-JSON]
    {{
      "answer": "두괄식 답변 및 논리적 근거 (각 근거 문장 끝에 [[n]])",
      "used_index": ["0", "3"]
    }}
    """

    prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_final)
    chain = prompt | self.llm | JsonOutputParser()
    answer = chain.invoke({"context": context, "question": self.query})

    used_docs = []
    for x in answer.pop("used_index"):
      try:
        if int(x) < len(reranked_docs):
          used_docs.append(reranked_docs[int(x)])
      except (ValueError, IndexError):
        continue

    answer_text = answer.pop("answer")

    return answer_text, used_docs, reranked_docs





In [6]:
class Reference:

  def strip_dot(self, text):
    if not text:
      return ""
    return text.rstrip(".")

  def clean_content(self, text):
    if not text:
      return text

    text = text.strip()

    text = re.sub(r"^[①②③④⑤⑥⑦⑧⑨⑩]+\s*", "", text)
    text = re.sub(r"^\d+\.\s*", "", text)
    text = re.sub(r"^[가-힣]\.\s*", "", text)

    return text.strip()

  def make_reference(self, doc, max_law_chars=300):
    출처 = ""
    meta = doc.metadata
    law_name = meta.get("law_name")
    출처 += law_name
    조문번호 = meta.get("조문번호", "")
    출처 += f" {조문번호}조" if 조문번호 else ""
    항번호 = meta.get("항번호", "")
    출처 += f" {self.strip_dot(항번호)}항" if 항번호 else ""
    호번호 = meta.get("호번호", "")
    출처 += f" {self.strip_dot(호번호)}호" if 호번호 else ""
    목번호 = meta.get("목번호", "")
    출처 += f" {self.strip_dot(목번호)}목" if 목번호 else ""

    content = self.clean_content(doc.page_content)
    if len(content) > max_law_chars:
      content = content[:max_law_chars] + "..."

    return 출처, content

  def generate_reference(self, pipeline):
    answer_text, used_docs, reranked_docs = pipeline.generate_answer()

    reference_list= []
    for doc in used_docs:
      출처, content = self.make_reference(doc)
      reference_list.append(f"{출처}: {content}")

    final_answer = answer_text + "\n\n[근거 법령 및 본문]" + "\n" .join(reference_list)

    return final_answer, reranked_docs


In [ ]:
from langchain_core.globals import set_debug

set_debug(False)

In [ ]:
query = "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"

answer, used_docs = Pipeline(query, llm, vectorstore).generate_answer()
print(answer)
print(used_docs)


KeyboardInterrupt: 

In [ ]:
query = "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"

pipeline = Pipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline))


[ID 0] ([출처] law name: 소득세법, 조문제목: 기본공제)
2. 거주자의 배우자로서 해당 과세기간의 소득금액이 없거나 해당 과세기간의 소득금액 합계액이 100만원 이하인 사람(총급여액 500만원 이하의 근로소득만 있는 배우자를 포함한다)

[ID 1] ([출처] law name: 소득세법 시행령, 조문제목: 주소와 거소의 판정)
①「소득세법」(이하 "법"이라 한다) 제1조의2에 따른 주소는 국내에서 생계를 같이 하는 가족 및 국내에 소재하는 자산의 유무등 생활관계의 객관적 사실에 따라 판정한다. <개정 2010.2.18>

[ID 2] ([출처] law name: 소득세법, 조문제목: 생계를 같이 하는 부양가족의 범위와 그 판정시기)
제53조(생계를 같이 하는 부양가족의 범위와 그 판정시기)

[ID 3] ([출처] law name: 법인세법, 조문제목: 과세소득의 범위)
3. 「소득세법」 제17조제1항에 따른 배당소득

[ID 4] ([출처] law name: 법인세법, 조문제목: 기부금영수증 발급ㆍ작성ㆍ보관 불성실 가산세)
2. 「소득세법」 제34조 및 제59조의4제4항에 따라 기부금을 필요경비에 산입하거나 기부금세액공제를 받기 위하여 필요한 영수증

[ID 5] ([출처] law name: 소득세법, 조문제목: 연금계좌세액공제)
① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 "연금계좌 납입액"이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 

소득금액에 대해 정확히 설명하기 위해서는 과세표준과 관련된 소득세법의 조문을 retrieve 해야 하는데\
reranking prompt에서 이를 위한 guidliine을 제시해도, 검색되어 context로 제공되는 문서들은 주로 '기본공제'에 대한 문서들임.


In [ ]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = Pipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline))


[ID 0] ([출처] law name: 법인세법, 조문제목: 중소기업의 결손금 소급공제에 따른 환급)
① 중소기업에 해당하는 내국법인은 각 사업연도에 결손금이 발생한 경우 대통령령으로 정하는 직전 사업연도의 법인세액(이하 이 조에서 "직전 사업연도의 법인세액"이라 한다)을 한도로 제1호의 금액에서 제2호의 금액을 차감한 금액을 환급 신청할 수 있다. <개정 2018.12.24>

[ID 1] ([출처] law name: 법인세법 시행령, 조문제목: 결손금공제)
②법 제13조제1항제1호에 따라 결손금을 공제할 때에는 먼저 발생한 사업연도의 결손금부터 차례대로 공제한다. <개정 2016.2.12, 2019.2.12>

[ID 2] ([출처] law name: 법인세법, 조문제목: 중소기업의 결손금 소급공제에 따른 환급)
2. 결손금이 발생한 사업연도의 직전 사업연도에 대한 법인세 과세표준과 세액을 제66조에 따라 경정함으로써 환급세액이 감소된 경우

[ID 3] ([출처] law name: 법인세법 시행령, 조문제목: 결손금공제)
3. 법 제72조제1항 및 「조세특례제한법」 제8조의4에 따라 공제받은 결손금

[ID 4] ([출처] law name: 소득세법 시행령, 조문제목: 중소기업 및 중견기업의 범위)
③ 제1항을 적용할 때 중소기업에 해당하는지 여부의 판정은 「중소기업기본법 시행령」 제3조의3제1항에도 불구하고 주식등의 양도일이 속하는 사업연도의 직전 사업연도 종료일 현재를 기준으로 한다. 다만, 주식등의 양도일이 속하는 사업연도에 새로 설립된 법인의 경우에는 주식등의 양도일 현재를 기준으로 한다.

[ID 5] ([출처] law name: 법인세법 시행령, 조문제목: 구분경리)
② 법 제113조제3항 각 호 외의 부분 단서 및 같은 조 제4항 각 호 외의 부분 단서에 따른 중소기업의 판정은 합병 또는 분할합병 전의 현황에 따르고, 동일사업을 영위하는 법인(분할법인의 경우 승계된 사업분에 한정한다)의 판정은 실질적으로 동일한 사업을 영위하는 것으로서 재정

In [ ]:
from datasets import Dataset
from google.colab import userdata

answers = []
contexts = []
questions = ["조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."]
ground_truth = ["결손금 소급공제에 의한 환급 적용 대상 법인은 조세특례제한법 시행령 제2조의 규정에 의한 중소기업으로서 결손금이 발생한 사업연도와 그 직전 사업연도의 법인세 과세표준과 세액을 법정 신고기한 내에 각각 신고한 경우에 한하여 적용하는 것으로 직전 사업연도에 중소기업에 해당하지 않아도 당해 사업연도에 중소기업에 해당하는 경우에는 당해 연도에 발생한 결손금에 대하여 결손금 소급공제를 적용받을 수 있는 것입니다."]

for query in questions:
  answer, _, reranked_docs = Pipeline(query,llm, vectorstore).generate_answer()
  answers.append(answer)
  contexts.append([doc.page_content for doc in reranked_docs])

data = {
    "user_input": questions,
    "response": answers,
    "retrieved_contexts": contexts,
    "reference": ground_truth
}

dataset = Dataset.from_dict(data)

try:
  from ragas import evaluate
except ImportError:
  !pip install ragas
  from ragas import evaluate

try:
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
except ImportError:
  !pip install ragas
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

llm_ragas = LangchainLLMWrapper(llm)
embedding_ragas = LangchainEmbeddingsWrapper(embeddings)

result = evaluate(dataset=dataset,
                  metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
                  llm=llm_ragas,
                  embeddings=embedding_ragas)

df = result.to_pandas()
df

/tmp/ipython-input-3939451623.py:30: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/tmp/ipython-input-3939451623.py:30: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/tmp/ipython-input-3939451623.py:30: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_recall
  from ragas.metrics import faithfulness, an

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision
0,조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직...,[① 중소기업에 해당하는 내국법인은 각 사업연도에 결손금이 발생한 경우 대통령령으로...,"네, 결손금이 발생한 연도에 중소기업에 해당하면 환급이 가능합니다.",결손금 소급공제에 의한 환급 적용 대상 법인은 조세특례제한법 시행령 제2조의 규정에...,1.0,0.498977,1.0,1.0


In [ ]:
{
        "query": "법인세 신고 후 법인통장으로 입금된 매출누락이 발견되어 수정신고를 하는 경우 소득처분은 어떻게 하여야 하나요?",
        "answer": "법인의 각 사업연도 소득금액 계산상 매출누락이 확인되는 경우 매출누락액이 실제 법인 계좌로 입금되어 선수금으로 처리되어 동 금액이 사외유출되지 아니하고 사내에 남아 있는 경우 익금산입(유보)으로 소득처분하고 이미 사외유출 된 금액을 회수한 경우도 동일하게 익금산입(유보)로 소득처분하는 것이나, 사외유출 기간 동안 가지급금으로 보아 인정이자를 계산하여야 하는 것입니다. 매출누락액이 사외 유출된 경우로써 그 귀속을 알 수 없는 경우에는 익금산입(대표자 상여)로 소득처분합니다. 그러나, 세무조사의 통지를 받거나 세무조사에 착수된 것을 알게 된 경우, 과세자료 소명 안내문을 받은 경우 등 경정이 있을 것을 미리 알고 익금산입하는 경우에는 소득의 귀속자에게 소득처분하는 것이며 귀속자가 불분명한 경우에는 대표이사에게 상여처분을 하는 것이며, 당해 매출누락금액이 법인통장에 입금되었다 할지라도 그 명목이 가수금의 입금 또는 가지급금의 회수, 선급금 또는 외상매출금의 회수 등으로 처리된 경우에는 해당 귀속자에게 소득처분 하여야 할 것입니다.",
        "gold": "법인세법 시행령 제106조 소득처분 ④",
        "source": "법인세법"
    },

In [7]:
query = "법인이 장기 미회수 매출채권에 대하여 일정부분을 감면해주고 현금으로 회수하기로 합의할 경우 감면해준 채무금액에 대하여 손금으로 인정받을 수 있나요?"

pipeline = Pipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline)[0])

감면해준 채무금액은 회수불능 사유에 해당하는 경우 손금으로 인정받을 수 있습니다.[[0]]

[근거 법령 및 본문]법인세법 시행규칙 10조 ①항 3호: 채무자의 인수거절ㆍ지급거절에 따라 채권금액의 회수가 불가능하거나 불가피하게 거래당사자 간의 합의에 따라 채권금액을 감면하기로 한 경우로서 이를 현지의 거래은행ㆍ검사기관ㆍ공증기관ㆍ공공기관 또는 해외채권추심기관이 확인하는 경우(채권금액의 일부를 감액한 경우에는 그 감액된 부분으로 한정한다)
법인세법 시행령 51조 ①항 2호: 채권자의 능력 및 자산상태로 보아 금전을 대여한 것으로 인정할 수 없는 차입금


In [8]:
from datasets import Dataset
from google.colab import userdata

answers = []
contexts = []
questions = ["법인이 장기 미회수 매출채권에 대하여 일정부분을 감면해주고 현금으로 회수하기로 합의할 경우 감면해준 채무금액에 대하여 손금으로 인정받을 수 있나요?"]
ground_truth = ["법인이 약정에 의하여 채권의 전부 또는 일부를 포기하는 경우 대손금으로 보지 아니하며 기부금 또는 접대비로 봅니다. 법인이 거래처와의 원활한 거래관계 유지 등 사업과 관련하여 채권을 포기하는 경우에는 접대비로 보아 한도내에서 손금에 산입하는 것이며, 그 외의 경우는 기부금으로 보는 것으로, 기부금영수증을 발급받지 아니한 경우에는 손금에 산입할 수 없습니다 (*거래처가 공익법인 등에 해당하여 기부금영수증을 받는 경우에는 기부금한도내에서 손금산입 가능) 다만, 특수관계자 외의 자와의 거래에서 발생한 채권으로서 채무자의 부도발생 등으로 장래에 회수가 불확실한 어음․수표상의 채권 등을 조기에 회수하기 위하여 일부를 불가피하게 포기한 경우 등 채권의 일부를 포기하거나 면제한 행위에 객관적으로 정당한 사유가 있는 때에는 채권포기액에 대하여 채권을 포기한 사업연도의 손금으로 산입할 수 있습니다."]

for query in questions:
  answer, _, reranked_docs = Pipeline(query,llm, vectorstore).generate_answer()
  answers.append(answer)
  contexts.append([doc.page_content for doc in reranked_docs])

data = {
    "user_input": questions,
    "response": answers,
    "retrieved_contexts": contexts,
    "reference": ground_truth
}

dataset = Dataset.from_dict(data)

try:
  from ragas import evaluate
except ImportError:
  !pip install ragas
  from ragas import evaluate

try:
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
except ImportError:
  !pip install ragas
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

llm_ragas = LangchainLLMWrapper(llm)
embedding_ragas = LangchainEmbeddingsWrapper(embeddings)

result = evaluate(dataset=dataset,
                  metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
                  llm=llm_ragas,
                  embeddings=embedding_ragas)

df = result.to_pandas()
df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.5/466.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.13.0
    Uninstalling jiter-0.13.0:
      Successfully uninstalled jiter-0.13.0


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-632762870.py:30: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/tmp/ipython-input-632762870.py:30: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.c

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision
0,법인이 장기 미회수 매출채권에 대하여 일정부분을 감면해주고 현금으로 회수하기로 합의...,[3. 채무자의 인수거절ㆍ지급거절에 따라 채권금액의 회수가 불가능하거나 불가피하게 ...,감면해준 채무금액은 회수불능 사유에 해당하는 경우 손금으로 인정받을 수 있습니다.[...,법인이 약정에 의하여 채권의 전부 또는 일부를 포기하는 경우 대손금으로 보지 아니하...,1.0,0.526225,0.666667,0.97619
